# RAG(검색-증강 생성) Retrival-Augmented Generation
- LLM에게 외부 데이터를 컨텍스트로 활용


In [13]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [14]:
%pip install -q "langchain_chroma>=0.1.2" langchain_community faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

# x = embedding -----------------
# print(len(x), x)

# VectorStore 에 임베딩 후 저장
vectorstore = Chroma.from_documents(
    documents,
    embedding=embedding
)


In [16]:
vectorstore.similarity_search(
    '초보자가 키우기 좋은 애완동물 추천해줘', k=2
)

[Document(id='7d6700d4-adfb-40b6-885c-be9efd10ac79', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(id='7e53cc47-de2f-4fce-b310-e67010b6051f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

## 사전처리 단계
1. 문서 불러오기 (Loading)
2. 텍스트 나누기  (Splitting)
3. 숫자로 바꾸기 (Embedding)
4. 저장하기 (VectorStore)

In [17]:
%pip install -q pymupdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('./spri.pdf')
docs = loader.load()


print('원본 pdf의 장수', len(docs))

원본 pdf의 장수 23


In [19]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

print('분할 후 청크 수', len(split_docs))

분할 후 청크 수 72


In [20]:
# 3. 임베딩, 4. 벡터스토어 저장
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# Test
vectorstore.similarity_search(
    '미국 대통령과 관련된 문서들 가져와봐', k=4
)

[Document(id='864d237b-07ef-440d-b3c4-46961d3b719f', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './spri.pdf', 'file_path': './spri.pdf', 'total_pages': 23, 'format': 'PDF 1.6', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2025-09-03T13:34:34+09:00', 'trapped': '', 'modDate': "D:20250903133434+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 3}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄'),
 Document(id='7

## 검색 증강 단계
1. 사용자 질문 (Query)
2. 겸색 (Retrieve)
3. LLM
4. 최종 답변

In [21]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain import hub

# Prompt 세팅
prompt = hub.pull('rlm/rag-prompt')

# LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('삼성전자 관련 소식 다 가져와')

'삼성전자는 자체 개발한 생성 AI ‘삼성 가우스’를 공개했으며, 온디바이스에서 작동 가능하고 언어, 코드, 이미지 모델로 구성되어 있습니다. 삼성 가우스는 안전한 데이터로 학습되었으며, 다양한 제품에 단계적으로 탑재될 예정입니다. 2023년 11월 열린 ‘삼성 AI 포럼 2023’에서 처음 선보였으며, 경쟁력 있는 AI 기술로 기대되고 있습니다.'

In [23]:
# Agent + RAG
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_tavily import TavilySearch
from datetime import datetime
# RAG 관련
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

today = datetime.today().strftime('%Y-%m-%d')

llm = ChatOpenAI(model='gpt-4.1-nano')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

loader = PyMuPDFLoader('./spri.pdf')
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding=embedding)
retriever = vectorstore.as_retriever()

rag_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색합니다.'  # Agent가 언제 이 tool을 쓸지 알게됨
)

text = f"""
너는 웹 검색이 가능하고, 2023년 12월 인공지능 산업 최신동향 정보를 담은 pdf 를 검색할 수 있는 어시스턴트야.

- 사용자가 PDF 문서와 관련된 질문(ex. '이 pdf에서', '문서내용', '파일에서)을 하면 반드시 'pdf_search' 도구를 써야해
- 사용자 질문이 팩트체크를 필요로 하고, 최신성이 필요하다 판단되면 web_search를 실행해야해
- 사용자가 일반적인 질문을 하고, 최신성이나 팩트체크가 필요없으면 그냥 답변해
- 뭔가 확실하지 않으면 pdf_search 와 web_search를 모두 실행해서 답변을 생성해

오늘은 {today} 야.
"""


prompt = ChatPromptTemplate.from_messages([
    ('system', text),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool, rag_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    memory=memory, 
    tools=[search_tool, rag_tool],
    verbose=True)

C:\Users\trueb\AppData\Local\Temp\ipykernel_10136\3201388536.py:56: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [24]:
agent_executor.invoke({'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '2023년 이후 인공지능 산업 최신 동향'}`


{'query': '2023년 이후 인공지능 산업 최신 동향', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'http://www.itdaily.kr/news/articleView.html?idxno=232711', 'title': '[초점] 안정기 접어든 국내 AI 산업, 2024년 6.3조원 시장 형성', 'content': '[아이티데일리] 2024년 국내 인공지능(AI) 기업들의 총 매출이 6조 3,009억 원을 기록했다. 2024년 국내 AI 산업 전체 매출은 6조 3,009억 원으로, 2023년 5조 6,991억 원 대비 7,018억 원(12.5%) 증가했다. AI 산업 실태조사에 따르면, 2024년 AI 사업 분야별로는 ‘AI 응용 SW’가 2조 6,682억 원(42.4%)로 전체 매출의 절반 가까이 차지했다. 클라우드 컴퓨팅, AI 컨설팅 등 인프라 지원 서비스를 중심으로 하는 AI 구축·관리 및 관련 정보 서비스가 1조 8,665억 원(29.6%)으로 두 번째로 큰 비중을 보였다. 2024년 AI 부문 연구개발 투자액은 3조 7,000억 원으로 전년(3조 2,000억 원) 대비 21.9% 증가했다. 이에 따라 국내 AI 산업계는 해외 시장 정보와 네트워크, 맞춤형 컨설팅, 현지화 지원, 투자 연계, 글로벌 마케팅 등 실질적이고 체감할 수 있는 지원이 필요하다고 주문한다. [[데이터센터①] 생성형 AI 시대, 국내 데이터센터 산업은 ‘경색’](/news/articleView.html?', 'score': 0.796562, 'raw_content': None}, {'url': 'https://nsp.nanet.go.kr/plan/subject/detail.

{'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?',
 'chat_history': [HumanMessage(content='방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2023년 이후 인공지능 산업은 지속적인 성장과 변화가 이루어지고 있습니다. 최근의 주요 동향을 살펴보면:\n\n1. **시장 확대**: 2024년 국내 AI 산업 매출이 6조 3,009억 원에 달하며, 2023년에 비해 12.5% 성장하였고, AI 응용 소프트웨어가 전체 매출의 주요 비중을 차지하고 있습니다.\n2. **기술 발전**: 생성형 AI와 컴퓨터 비전 등 신기술이 산업 전반에서 활용도가 높아지고 있으며, 제조업, 헬스케어, 데이터 센터 등 다양한 분야에서의 적용이 활발하게 일어나고 있습니다.\n3. **글로벌 시장**: 글로벌 인공지능 시장은 2025년 약 294억 달러에 달하며, 연평균 성장률이 29.2%에 해당하는 높은 상승세를 보이고 있습니다.\n4. **정책 및 규제**: AI 윤리와 규제의 중요성이 부상하며, 정부와 기업들이 보안, 윤리, 정책 대응 전략에 더욱 집중하고 있습니다.\n5. **핵심 트렌드**: 생성 AI의 폭발적 성장, AI 윤리와 규제, 헬스케어 융합, 엣지 AI와 분산 컴퓨팅이 주목받고 있습니다.\n\n이와 같이 2023년 이후로 인공지능 산업은 기술적 진보와 함께 시장 규모의 확대, 정책적 대응 등 다양한 흐름이 활발하게 진행되고 있습니다.', additional_kwargs={}, response_metadata={})],
 'output': '2023년 이후 인공지능 산업은 지속적인 성장과 변화가 이루어지고 있습니다. 최근의 주요 동향을 살펴보면:\n\n1. **시장 확대**: 2024년 국내 AI 산업 매출이 6조 3,009억 원에 달하며, 2023년에 비해 12.5% 성장하였고,

In [25]:
pip install chainlit

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   -------


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
